# ETL to Retrieve Demographics Data from private API

### NOTE: Leave numeric values as INT - convert to percentage on collapse

In [1]:
import pandas as pd

In [2]:
complete = pd.read_csv('../data_files/bls_metro_2020_clean.csv', header=0)

## Unique metro points

In [3]:
complete_unique_citi_state = complete['area_title'].unique()
complete_unique_citi_state.shape

(396,)

### Dataframes

In [4]:
columns = ['area_title', 'area_name', 'zipcode', 'state', 'city', 'population', 'median_age', 'race_asian', 'race_white', 'race_black', 'race_native', 'race_islander', 'race_other', 'race_two', 'race_hispanic', 'average_household_income', 'family_households_total', 'family_poverty_pct', 'educational_attainment_bachelors', 'educational_attainment_graduate' , 'educational_attainment_high_school', 'educational_attainment_no_diploma', 'educational_attainment_some_college' ]
demographic_by_zipcode_df = pd.DataFrame(columns = columns)

In [5]:
# Lookup dict, convert metro state abbreviation to full string
# We have abbreviations, database expects full state name
states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'}

In [6]:
# stores any location with issues
# Next phase of project
# will circle back to locations with multiple city/state concatenated
location_issues_df = pd.DataFrame(columns=['area_name', 'city', 'state'])

## Start request to private db

In [7]:
# Intentionally left out credentials
# will update with config next commit
from config import p
import requests
import mysql.connector
servername = "internal-db.s221289.gridserver.com"
username = "db221289_stp"
password = ""
dbname = "db221289_stp"
cnx = mysql.connector.connect(user=username, password=password,host=servername,database=dbname)
cursor = cnx.cursor()

### Call to database

In [8]:
def requestDemographicForCityState(_area_name, _city, _state):
    # check if this state exists (seeing some strange state abbrs)
    if _state.upper() in states :
        search_state =  states[_state.upper()].upper()
        search_city =  _city.upper().replace("'", "''")
        query = (f"SELECT '{search_city}' AS area_title, zip_zcta, state_name, city_name, age_total as population, median_age, race_and_ethnicity_asian, race_and_ethnicity_white, race_and_ethnicity_black, race_and_ethnicity_native, race_and_ethnicity_islander, race_and_ethnicity_other, race_and_ethnicity_two, race_and_ethnicity_hispanic, average_household_income, family_households_total, family_poverty_pct, educational_attainment_bachelors, educational_attainment_graduate , educational_attainment_high_school, educational_attainment_no_diploma, educational_attainment_some_college FROM zipcodeDemographic WHERE state_name = '{search_state}' and city_name LIKE '%%{search_city}%%' ")
        cursor.execute(query)
        # iterate throw rows and add to df
        for zipcode in cursor:
            #print (zipcode.dtype)
            newrow = (_area_name,) + zipcode
            demographic_by_zipcode_df.loc[len(demographic_by_zipcode_df.index)] = newrow
    
# iterate through locations list
for location in complete_unique_citi_state:
    locationParts = location.split(', ')
    locationStateParts = locationParts[1].split('-')
    # temp check to make sure the stateParts array only contain 1 value
    # if state array is more than 1 value then add to location_issues_df for later parsing
    if len(locationStateParts) > 1:
        print (f"Issues with {location}")
        #ocation_issues_df.loc[len(location_issues_df.index)] = [location['city'], location['state']]
        location_issues_df.loc[len(location_issues_df.index)] = [location, locationParts[0], locationParts[1]]
    # else lets make that request
    else:
        city = locationParts[0]
        state = locationParts[1]
        requestDemographicForCityState(location, city, state)
        

Issues with Allentown-Bethlehem-Easton, PA-NJ
Issues with Augusta-Richmond County, GA-SC
Issues with Cape Girardeau, MO-IL
Issues with Charlotte-Concord-Gastonia, NC-SC
Issues with Chattanooga, TN-GA
Issues with Chicago-Naperville-Elgin, IL-IN-WI
Issues with Cincinnati, OH-KY-IN
Issues with Clarksville, TN-KY
Issues with Columbus, GA-AL
Issues with Cumberland, MD-WV
Issues with Davenport-Moline-Rock Island, IA-IL
Issues with Duluth, MN-WI
Issues with Evansville, IN-KY
Issues with Fargo, ND-MN
Issues with Fayetteville-Springdale-Rogers, AR-MO
Issues with Fort Smith, AR-OK
Issues with Grand Forks, ND-MN
Issues with Hagerstown-Martinsburg, MD-WV
Issues with Huntington-Ashland, WV-KY-OH
Issues with Kansas City, MO-KS
Issues with Kingsport-Bristol-Bristol, TN-VA
Issues with La Crosse-Onalaska, WI-MN
Issues with Lewiston, ID-WA
Issues with Logan, UT-ID
Issues with Louisville/Jefferson County, KY-IN
Issues with Memphis, TN-MS-AR
Issues with Minneapolis-St. Paul-Bloomington, MN-WI
Issues with 

### Demographic by zipcode

In [9]:
demographic_by_zipcode_df.columns

Index(['area_title', 'area_name', 'zipcode', 'state', 'city', 'population',
       'median_age', 'race_asian', 'race_white', 'race_black', 'race_native',
       'race_islander', 'race_other', 'race_two', 'race_hispanic',
       'average_household_income', 'family_households_total',
       'family_poverty_pct', 'educational_attainment_bachelors',
       'educational_attainment_graduate', 'educational_attainment_high_school',
       'educational_attainment_no_diploma',
       'educational_attainment_some_college'],
      dtype='object')

In [10]:
demographic_by_zipcode_df.dtypes

area_title                             object
area_name                              object
zipcode                                object
state                                  object
city                                   object
population                             object
median_age                             object
race_asian                             object
race_white                             object
race_black                             object
race_native                            object
race_islander                          object
race_other                             object
race_two                               object
race_hispanic                          object
average_household_income               object
family_households_total                object
family_poverty_pct                     object
educational_attainment_bachelors       object
educational_attainment_graduate        object
educational_attainment_high_school     object
educational_attainment_no_diploma 

### Strip the '%'

In [11]:
demographic_by_zipcode_df['family_poverty_pct'] = demographic_by_zipcode_df['family_poverty_pct'].str.replace('%', '')
demographic_by_zipcode_df['educational_attainment_bachelors'] = demographic_by_zipcode_df['educational_attainment_bachelors'].map(lambda x: x.strip('%'))
demographic_by_zipcode_df['educational_attainment_graduate'] = demographic_by_zipcode_df['educational_attainment_graduate'].map(lambda x: x.strip('%'))
demographic_by_zipcode_df['educational_attainment_high_school'] = demographic_by_zipcode_df['educational_attainment_high_school'].map(lambda x: x.strip('%'))
demographic_by_zipcode_df['educational_attainment_no_diploma'] = demographic_by_zipcode_df['educational_attainment_no_diploma'].map(lambda x: x.strip('%'))
demographic_by_zipcode_df['educational_attainment_some_college'] = demographic_by_zipcode_df['educational_attainment_some_college'].map(lambda x: x.strip('%'))
demographic_by_zipcode_df

,area_title,area_name,zipcode,state,city,population,median_age,race_asian,race_white,race_black,...,race_two,race_hispanic,average_household_income,family_households_total,family_poverty_pct,educational_attainment_bachelors,educational_attainment_graduate,educational_attainment_high_school,educational_attainment_no_diploma,educational_attainment_some_college
0,"Abilene, TX",ABILENE,79563,Texas,Tye; Abilene,1052,33,0,756,11,...,12,228,45376,291,8.9,7.9,0.9,50.8,18.1,22.3
1,"Abilene, TX",ABILENE,79601,Texas,Abilene,28041,30,546,15155,3887,...,693,7323,52921,4321,12.8,10.9,6.9,35.9,19.7,26.6
2,"Abilene, TX",ABILENE,79602,Texas,Abilene; Potosi,24027,33.3,206,16808,1265,...,496,5133,78945,5937,9.4,18.9,8.9,26.8,12.4,32.9
3,"Abilene, TX",ABILENE,79603,Texas,Impact; Abilene; Tye,23577,32.9,167,10196,2402,...,208,10545,51676,5840,19.0,11.3,2.6,35.7,22.7,27.7
4,"Abilene, TX",ABILENE,79605,Texas,Abilene,29672,33.6,569,19773,2168,...,467,6421,63713,7343,12.2,14.1,6.9,31.5,12.2,35.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026,"Waterbury, CT",WATERBURY,06706,Connecticut,Waterbury,15016,33.2,630,5285,1921,...,229,6832,55294,3544,21.4,9.6,6.0,35.3,24.4,24.7
2027,"Waterbury, CT",WATERBURY,06708,Connecticut,Watertown; Waterbury,28654,34.5,487,15691,3949,...,826,7219,71618,6581,16.8,14.7,10.0,36.5,13.5,25.3
2028,"Waterbury, CT",WATERBURY,06710,Connecticut,Waterbury,10803,28.5,272,2430,2187,...,200,5597,43763,2145,26.1,10.6,5.2,37.0,25.1,22.2
2029,"Waterbury, CT",WATERBURY,06716,Connecticut,Waterbury; Wolcott,16696,45.2,226,15254,387,...,127,701,97772,4480,4.3,17.4,11.8,35.9,7.2,27.8


### Convert objects to int

In [12]:
convert_columns_int = [ 'population', 'race_asian', 'race_white', 'race_black', 'race_native', 'race_islander', 'race_other', 'race_two', 'race_hispanic', 'average_household_income', 'family_households_total']
demographic_by_zipcode_df[convert_columns_int] = demographic_by_zipcode_df[convert_columns_int].astype(str).astype(int)

### Calculate columns with percentage values to numeric by population

Will calculate to percentage on collapse

In [13]:
demographic_by_zipcode_df['median_age'] = demographic_by_zipcode_df['median_age'].astype(float).astype(int)
demographic_by_zipcode_df['average_household_income'] = demographic_by_zipcode_df['average_household_income'].astype(float).astype(int)
demographic_by_zipcode_df['family_poverty'] = ((demographic_by_zipcode_df['family_poverty_pct'].astype("float") * demographic_by_zipcode_df['population']) / 100).astype(int)
demographic_by_zipcode_df['educational_attainment_bachelors'] = ((demographic_by_zipcode_df['educational_attainment_bachelors'].astype("float") * demographic_by_zipcode_df['population']) / 100).astype(int)
demographic_by_zipcode_df['educational_attainment_graduate'] = ((demographic_by_zipcode_df['educational_attainment_graduate'].astype("float") * demographic_by_zipcode_df['population']) / 100).astype(int)
demographic_by_zipcode_df['educational_attainment_high_school'] = ((demographic_by_zipcode_df['educational_attainment_high_school'].astype("float") * demographic_by_zipcode_df['population']) / 100).astype(int)
demographic_by_zipcode_df['educational_attainment_no_diploma'] = ((demographic_by_zipcode_df['educational_attainment_no_diploma'].astype("float") * demographic_by_zipcode_df['population']) / 100).astype(int)
demographic_by_zipcode_df['educational_attainment_some_college'] = ((demographic_by_zipcode_df['educational_attainment_some_college'].astype("float") * demographic_by_zipcode_df['population']) / 100).astype(int)

demographic_by_zipcode_df.head(2)


,area_title,area_name,zipcode,state,city,population,median_age,race_asian,race_white,race_black,...,race_hispanic,average_household_income,family_households_total,family_poverty_pct,educational_attainment_bachelors,educational_attainment_graduate,educational_attainment_high_school,educational_attainment_no_diploma,educational_attainment_some_college,family_poverty
0,"Abilene, TX",ABILENE,79563,Texas,Tye; Abilene,1052,33,0,756,11,...,228,45376,291,8.9,83,9,534,190,234,93
1,"Abilene, TX",ABILENE,79601,Texas,Abilene,28041,30,546,15155,3887,...,7323,52921,4321,12.8,3056,1934,10066,5524,7458,3589


In [14]:
demographic_by_zipcode_df.head(2)

,area_title,area_name,zipcode,state,city,population,median_age,race_asian,race_white,race_black,...,race_hispanic,average_household_income,family_households_total,family_poverty_pct,educational_attainment_bachelors,educational_attainment_graduate,educational_attainment_high_school,educational_attainment_no_diploma,educational_attainment_some_college,family_poverty
0,"Abilene, TX",ABILENE,79563,Texas,Tye; Abilene,1052,33,0,756,11,...,228,45376,291,8.9,83,9,534,190,234,93
1,"Abilene, TX",ABILENE,79601,Texas,Abilene,28041,30,546,15155,3887,...,7323,52921,4321,12.8,3056,1934,10066,5524,7458,3589


### Remove areas with population 0

In [15]:
demographic_by_zipcode_df = demographic_by_zipcode_df[demographic_by_zipcode_df['population'] > 0]
demographic_by_zipcode_df.shape

(1998, 24)

### Group zipcode demographics by city and state

NEED TO REWORK AGGREGATE

In [16]:
grouped_demographic = demographic_by_zipcode_df.groupby(['area_title'], as_index=False).agg(
    area_name = ('area_name', lambda x: x.iloc[0]),
    population = ('population', 'sum'),
    median_age = ('median_age', 'mean'),
    average_income = ('average_household_income', 'mean'),
    family_poverty = ('family_poverty', 'sum'),
    educational_attainment_bachelors = ('educational_attainment_bachelors', 'sum'),
    educational_attainment_graduate = ('educational_attainment_graduate', 'sum'),
    educational_attainment_high_school = ('educational_attainment_high_school', 'sum'),
    educational_attainment_no_diploma = ('educational_attainment_no_diploma', 'sum'),
    educational_attainment_some_college = ('educational_attainment_some_college', 'sum'),
    race_asian = ('race_asian', 'sum'),
    race_white = ('race_white', 'sum'),
    race_black = ('race_black', 'sum'),
    race_hispanic = ('race_hispanic', 'sum'),
    race_native = ('race_native', 'sum'),
    race_islander = ('race_islander', 'sum'),
    race_two = ('race_two', 'sum'),
    race_other = ('race_other', 'sum'))
grouped_demographic['average_income'] = grouped_demographic['average_income'].astype(float).astype(int)
grouped_demographic['median_age'] = grouped_demographic['median_age'].astype(int)

grouped_demographic

,area_title,area_name,population,median_age,average_income,family_poverty,educational_attainment_bachelors,educational_attainment_graduate,educational_attainment_high_school,educational_attainment_no_diploma,educational_attainment_some_college,race_asian,race_white,race_black,race_hispanic,race_native,race_islander,race_two,race_other
0,"Abilene, TX",ABILENE,133731,29,52989,16198,19975,9695,40965,19258,43669,2626,81145,12275,34147,689,73,2576,200
1,"Akron, OH",AKRON,374675,38,57023,52405,60910,29186,132252,41708,110658,9945,279041,66305,7245,784,77,10800,478
2,"Albany, GA",ALBANY,98002,35,53876,24957,11517,8460,29032,16919,32046,959,28981,63178,2916,119,26,1761,62
3,"Albany, OR",ALBANY,63503,38,72395,6409,11043,5806,15482,5150,26013,1185,52553,367,7282,410,47,1649,10
4,"Albuquerque, NM",ALBUQUERQUE,638803,36,66708,92558,111538,91807,152964,78799,203804,14485,252843,16466,317922,22924,353,11870,1940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,"Wilmington, NC",WILMINGTON,203209,38,73390,24661,52916,26429,44028,15960,63855,2824,155339,28915,11206,563,151,3894,317
220,"Winston-Salem, NC",WINSTON-SALEM,329113,36,62399,45209,67318,37479,84864,37954,101423,6948,193899,82010,38918,508,165,5574,1091
221,"Yakima, WA",YAKIMA,145988,35,66839,19693,16594,10781,40498,31723,46357,1392,84786,1349,53274,1791,109,3207,80
222,"Yuba City, CA",YUBA CITY,77671,36,71409,11213,9668,4388,19203,16067,28376,13478,35947,1668,22226,459,324,3406,163


### Convert columns previously converted to int values back to % based on area_name's population

In [17]:
#grouped_demographic['family_poverty_pct'] = (grouped_demographic['family_poverty'] / grouped_demographic['population']) * 100
#grouped_demographic['educational_attainment_bachelors'] = (grouped_demographic['educational_attainment_bachelors'] / grouped_demographic['population']) * 100
#grouped_demographic['educational_attainment_graduate'] = (grouped_demographic['educational_attainment_graduate'] / grouped_demographic['population']) * 100
#grouped_demographic['educational_attainment_high_school'] = (grouped_demographic['educational_attainment_high_school'] / grouped_demographic['population']) * 100
#grouped_demographic['educational_attainment_no_diploma'] = (grouped_demographic['educational_attainment_no_diploma'] / grouped_demographic['population']) * 100
#grouped_demographic['educational_attainment_some_college'] = (grouped_demographic['educational_attainment_some_college'] / grouped_demographic['population']) * 100


grouped_demographic

,area_title,area_name,population,median_age,average_income,family_poverty,educational_attainment_bachelors,educational_attainment_graduate,educational_attainment_high_school,educational_attainment_no_diploma,educational_attainment_some_college,race_asian,race_white,race_black,race_hispanic,race_native,race_islander,race_two,race_other
0,"Abilene, TX",ABILENE,133731,29,52989,16198,19975,9695,40965,19258,43669,2626,81145,12275,34147,689,73,2576,200
1,"Akron, OH",AKRON,374675,38,57023,52405,60910,29186,132252,41708,110658,9945,279041,66305,7245,784,77,10800,478
2,"Albany, GA",ALBANY,98002,35,53876,24957,11517,8460,29032,16919,32046,959,28981,63178,2916,119,26,1761,62
3,"Albany, OR",ALBANY,63503,38,72395,6409,11043,5806,15482,5150,26013,1185,52553,367,7282,410,47,1649,10
4,"Albuquerque, NM",ALBUQUERQUE,638803,36,66708,92558,111538,91807,152964,78799,203804,14485,252843,16466,317922,22924,353,11870,1940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,"Wilmington, NC",WILMINGTON,203209,38,73390,24661,52916,26429,44028,15960,63855,2824,155339,28915,11206,563,151,3894,317
220,"Winston-Salem, NC",WINSTON-SALEM,329113,36,62399,45209,67318,37479,84864,37954,101423,6948,193899,82010,38918,508,165,5574,1091
221,"Yakima, WA",YAKIMA,145988,35,66839,19693,16594,10781,40498,31723,46357,1392,84786,1349,53274,1791,109,3207,80
222,"Yuba City, CA",YUBA CITY,77671,36,71409,11213,9668,4388,19203,16067,28376,13478,35947,1668,22226,459,324,3406,163


### Drop columns only used for calculations or no longer needed

In [18]:
#grouped_demographic.drop(['family_poverty'], axis=1, inplace=True)
grouped_demographic['area_name'] = grouped_demographic['area_title']
grouped_demographic.drop(columns=['area_title'])

,area_name,population,median_age,average_income,family_poverty,educational_attainment_bachelors,educational_attainment_graduate,educational_attainment_high_school,educational_attainment_no_diploma,educational_attainment_some_college,race_asian,race_white,race_black,race_hispanic,race_native,race_islander,race_two,race_other
0,"Abilene, TX",133731,29,52989,16198,19975,9695,40965,19258,43669,2626,81145,12275,34147,689,73,2576,200
1,"Akron, OH",374675,38,57023,52405,60910,29186,132252,41708,110658,9945,279041,66305,7245,784,77,10800,478
2,"Albany, GA",98002,35,53876,24957,11517,8460,29032,16919,32046,959,28981,63178,2916,119,26,1761,62
3,"Albany, OR",63503,38,72395,6409,11043,5806,15482,5150,26013,1185,52553,367,7282,410,47,1649,10
4,"Albuquerque, NM",638803,36,66708,92558,111538,91807,152964,78799,203804,14485,252843,16466,317922,22924,353,11870,1940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,"Wilmington, NC",203209,38,73390,24661,52916,26429,44028,15960,63855,2824,155339,28915,11206,563,151,3894,317
220,"Winston-Salem, NC",329113,36,62399,45209,67318,37479,84864,37954,101423,6948,193899,82010,38918,508,165,5574,1091
221,"Yakima, WA",145988,35,66839,19693,16594,10781,40498,31723,46357,1392,84786,1349,53274,1791,109,3207,80
222,"Yuba City, CA",77671,36,71409,11213,9668,4388,19203,16067,28376,13478,35947,1668,22226,459,324,3406,163


In [19]:
# demograhpics by zipcode to csv
demographic_by_zipcode_df.to_csv('../data_files/metro_demographic_by_zipcode.csv') 

In [20]:
# write to csv
grouped_demographic.to_csv('../data_files/metro_demographic_by_city_state.csv') 

In [21]:
# shut it down
cnx.close()